In [1]:
using NLsolve, Plots, Parameters, Distributions, Statistics, StatsPlots, Interpolations
include("discrete_normal.jl")
include("lininterp1.jl")

@with_kw struct Calibration
    #Households
    σ::Float64=0.5                      # elasticity of relative demand with respect to price sigma=0.5 completementary goods
    ζ::Float64=2.0                      # Intertemporal elasticity of substitution
    θ::Float64=1/ζ                      # Inverse of intertemporal elasticity of substitution 
    γ::Array{Float64}=[1/3, 1/3, 1/3]   # intensity in each good
    ϵ::Array{Float64}=[0.6, 1.0, 1.65]  # elasticity of relative demand with respect to income in luxury good sector
    ε::Array{Float64}=[ϵ[1]/ϵ[2], ϵ[2]/ϵ[2], ϵ[3]/ϵ[2] ]
    ρ::Float64 =(σ-1)/σ                 
    β::Float64 = 0.96                   # Discount factor

    #Production
    α::Float64=0.4                      # Capital share
    δ::Float64=0.1                      # Capital depreciation
    Z::Array{Float64}=[0.9, 1.0, 1.2]   # Sector productivity
end
cal = Calibration()

Calibration
  σ: Float64 0.5
  ζ: Float64 2.0
  θ: Float64 0.5
  γ: Array{Float64}((3,)) [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
  ϵ: Array{Float64}((3,)) [0.6, 1.0, 1.65]
  ε: Array{Float64}((3,)) [0.6, 1.0, 1.65]
  ρ: Float64 -1.0
  β: Float64 0.96
  α: Float64 0.4
  δ: Float64 0.1
  Z: Array{Float64}((3,)) [0.9, 1.0, 1.2]


In [2]:
@unpack σ, ζ, θ, γ, ϵ, ρ, β, α, δ, Z = cal
if θ==1
    u(c) = log.(c)
else
    u(c) = (c.^(1-θ).-1)./(1-θ)
end 

u1(c) = c.^(-θ)
u1inv(u) = u.^(-1 ./(θ))

u1inv (generic function with 1 method)

In [63]:
## income risk: discretized N(mu,sigma^2)
mu_y = 1
sd_y = 0.2
ny = 10

## asset grids
na = 40
amax = 50
borrow_lim = 0
agrid_par = 0.5 # 1 for linear, 0 for L-shaped

## computation
max_iter = 1000
tol_iter = 1.0e-4
Nsim = 50000
Tsim = 500

# OPTIONS
Display = 1
DoSimulate = 1
MakePlots = 1

## tolerance for non-linear solver
TolX = 1.0e-2

#comutation KL
maxiter_KL = 70
tol_KL = 1.0e-2
step_KL = 0.005
rguess = 1/β-1-0.001 # a bit lower than inverse of discount rate
KLratioguess = ((rguess + δ)/α)^(1/(α-1))

# SET UP GRIDS

## assets
agrid = range(0,1,length=na)
agrid = agrid.^(1 ./ agrid_par)
agrid = borrow_lim .+ (amax.-borrow_lim).*agrid

## income: disretize normal distribution
width = nlsolve(x -> discrete_normal(ny,mu_y,sd_y,x...)[1],[2.0]).zero
temp, ygrid, ydist = discrete_normal(ny,mu_y,sd_y,width...)
ycumdist = cumsum(ydist)

# DRAW RANDOM NUMBERS
yrand = rand(Nsim,Tsim)
yindsim = zeros(Int,Nsim,Tsim)

#Firms market value
KLratio = KLratioguess
r = α.*KLratio^(α-1) - δ
R = 1+r
wage = (1-α).* KLratio^α

1.2042882446158303

In [64]:
function solvingNHEE(u, cash::Float64, p_c::Vector{Float64}; cal=cal)
    @unpack σ, ζ, γ, ϵ, ρ, β, α, δ, Z = cal
    out=1-sum(γ[i]^(1/σ)*(((p_c[i]/cash)^(-σ)*γ[i])/u^((1-σ)*σ*ϵ[i]))^ρ for i=1:length(p_c))
    return out
end
function NHUtilityEE(cash, p_c::Vector{Float64}; cal=cal)
    @unpack σ, ζ, γ, ϵ, ρ, β, α, δ, Z = cal

    #Utility level for given C endowment
    res = nlsolve(u->[solvingNHEE(u[1], cash, p_c)], [1.0], xtol=TolX)
    U=res.zero[1]

    #Wealth and Minimum expenditure for the utility at given endowment level
    E=cash
    ExpNH=sum(γ[i]*U^(ϵ[i]*(1-σ)^2) * (p_c[i]^(1-σ)) for i = 1:length(p_c))^(1/(1-σ))
    
    #Optimal Demand with income effect
    C1=γ[1]*U^(ϵ[1]*(1-σ)^2) * (p_c[1]/E)^(-σ)
    C2=γ[2]*U^(ϵ[2]*(1-σ)^2) * (p_c[2]/E)^(-σ)
    C3=γ[3]*U^(ϵ[3]*(1-σ)^2) * (p_c[3]/E)^(-σ)

    #Expenditure share with income effect
    ω_p=γ[1]*U^(ϵ[1]*(1-σ)^2) * (p_c[1]/E)^(1-σ)
    ω_n=γ[2]*U^(ϵ[2]*(1-σ)^2) * (p_c[2]/E)^(1-σ)
    ω_l=γ[3]*U^(ϵ[3]*(1-σ)^2) * (p_c[3]/E)^(1-σ)

    ω=[ω_p ω_n ω_l]
    return [C1, C2, C3, U, E, ω, ExpNH]#, U, E, ω]

end

NHUtilityEE (generic function with 1 method)

In [65]:
p=[0.5, 1.0, 2.0]
cash=zeros(na,ny)
for i=1:ny
    cash[:,i]=r.*agrid.+wage.*ygrid[i]
end
cash

Uguess=zeros(na,ny)
cashU=zeros(na,ny)
for j=1:ny
    for i=1:na
        Uguess[i,j]=NHUtilityEE(cash[i,j], p)[4]
        cashU[i,j]=NHUtilityEE(cash[i,j], p)[7]
    end
end

In [67]:
eulerdiff(a)=u1(NHUtilityEE(cash.-a, p)[4])-β.*R.*(u1([lininterp1(agrid,Ulast[:,iy],a) for iy=1:ny])'*ydist)[1]


eulerdiff (generic function with 1 method)

In [68]:
iter = 0
cdiff = 1000
U = copy(Uguess)
global emuc = u1(U)*ydist

while iter<=max_iter && cdiff>tol_iter
    iter = iter + 1
    global Ulast = copy(U)
    global Elast = copy(cashU)
    global sav = zeros(na,ny)
    
    ## loop over assets
    for ia = 1:na     
        ## loop over income
        for iy = 1:ny
            global cash = R.*agrid[ia] .+ wage.*ygrid[iy]
                if u1(cash-borrow_lim) >= β.*R.*lininterp1(agrid,emuc,borrow_lim) # check if borrowing constrained
                    sav[ia,iy] = borrow_lim
                else
                    sav[ia,iy] = nlsolve(x -> [u1(NHUtilityEE(cash.-x[1], p)[4]).-β.*R.*lininterp1(agrid,emuc,x[1])], [cash.-Elast[ia,iy]]).zero[1]
                end
            U[ia,iy] = NHUtilityEE(cash .- sav[ia,iy],p)[4]
        end
    end

    emuc = u1(U)*ydist
    
    cdiff = maximum(abs.(U-Ulast))
    println("Iteration no. " * string(iter), " max con fn diff is " * string(cdiff))
end

Iteration no. 1 max con fn diff is 1.6839105858442336
Iteration no. 2

 max con fn diff is 0.0974631810918396


Iteration no. 3 max con fn diff is 0.016602594948227578
Iteration no. 4

 max con fn diff is 0.011347344407806048


Iteration no. 5 max con fn diff is 0.010896392130086419
Iteration no. 6

...

 max con fn diff is 0.00011656809080839992


Iteration no. 115 max con fn diff is 0.00011185975204419663
Iteration no. 116

 max con fn diff is 0.0001073420595210095


Iteration no. 117 max con fn diff is 0.00010300726734957522
Iteration no. 118

 max con fn diff is 9.884794864678526e-5


In [98]:
yindsim = zeros(Int,Nsim,Tsim)
asim = zeros(Nsim,Tsim)

## create interpolating function
savinterp = Array{Any}(undef,ny)
for iy = 1:ny
    savinterp[iy] = interpolate((agrid,), sav[:,iy], Gridded(Linear()))
end

In [101]:
if DoSimulate==1

    yindsim = zeros(Int,Nsim,Tsim)
    asim = zeros(Nsim,Tsim)
    
    ## create interpolating function
    savinterp = Array{Any}(undef,ny)
    for iy = 1:ny
        savinterp[iy] = interpolate((agrid,), sav[:,iy], Gridded(Linear()))
    end
    
    ## loop over time periods
    for it = 1:Tsim
        if Display >=1 && mod(it,100)==0
            println("Simulating, time period " * string(it))
        end
        
        ## income realization: note we vectorize simulations at once because
        ## of matlab, in other languages we would loop over individuals
        yindsim[yrand[:,it].<=ycumdist[1],it] .= 1
        for iy = 2:ny
            yindsim[(yrand[:,it].> ycumdist[iy-1]) .& (yrand[:,it].<=ycumdist[iy]),it] .= iy
        end
        
        ## asset choice
        if it<Tsim
            for iy = 1:ny
                asim[yindsim[:,it].==iy,it+1] = savinterp[iy](asim[yindsim[:,it].==iy,it])
            end
        end
    end
    
    ## assign actual income values
    ysim = ygrid[yindsim]

end

BoundsError: BoundsError: attempt to access 40-element interpolate((::Vector{Float64},), ::Vector{Float64}, Gridded(Linear())) with element type Float64 at index [[0.0, 0.2599282105041987, -0.029979000095993746, 0.15638593339394866, 0.3693012463585231, 0.0, 0.15638593339394866, 0.2599282105041987, 0.2599282105041987, 0.5974692524046055, 0.15638593339394866, 0.2599282105041987, 0.3693012463585231, 0.15638593339394866, 0.0618122940072156, 0.0, -0.029979000095993746, 0.0, 0.0618122940072156, 0.5974692524046055, 0.0618122940072156, 0.0, 0.0, 0.0618122940072156, 0.0618122940072156, -0.029979000095993746, 0.5974692524046055, 0.15638593339394866, 0.0, 0.3693012463585231, 0.2599282105041987, 0.0618122940072156, 0.3693012463585231, 0.15638593339394866, -0.029979000095993746, 0.0, 0.0618122940072156, 0.2599282105041987, 0.3693012463585231, 0.3693012463585231, 0.0, 0.4817878708095713, 0.0618122940072156, 0.0, 0.3693012463585231, 0.0618122940072156, 0.15638593339394866, 0.3693012463585231, 0.06181